<a href="https://colab.research.google.com/github/Ashwin1234/ML_OCR_project1/blob/main/HandWritten_OCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>